# angewandter Fall: HSozKult-Scraper

Ziel: Hsozkult scrapen

In [1]:
import re
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [2]:
base_url = "http://www.hsozkult.de"
query_string = "/searching/page?q="
search_string = "rezension grundwissenschaft"
search_string = search_string.replace(" ", "+")
url = base_url + query_string + search_string
url

'http://www.hsozkult.de/searching/page?q=rezension+grundwissenschaft'

In [3]:
search = urlopen(url)
bs = BeautifulSoup(search, 'html.parser')

In [4]:
results = bs.find_all('div', {'class': 'hfn-list-itemtitle'})
results

[<div class="hfn-list-itemtitle"><a href="/searching/id/rezbuecher-28479?title=t-kahlert-unternehmungen-grossen-stils&amp;q=rezension grundwissenschaft&amp;sort=&amp;fq=&amp;total=19&amp;recno=1&amp;subType=reb">				T. Kahlert: Unternehmungen großen Stils</a></div>,
 <div class="hfn-list-itemtitle"><a href="/searching/id/rezbuecher-27442?title=m-schroeter-erfolgreich-recherchieren-altertumswissenschaften&amp;q=rezension grundwissenschaft&amp;sort=&amp;fq=&amp;total=19&amp;recno=2&amp;subType=reb">				M. Schröter: Erfolgreich recherchieren – Altertumswissenschaften</a></div>,
 <div class="hfn-list-itemtitle"><a href="/searching/id/termine-34040?title=quellen-und-methoden-der-geschichtswissenschaft-im-digitalen-zeitalter-neue-zugaenge-fuer-eine-etablierte-disziplin-digimet-2017&amp;q=rezension grundwissenschaft&amp;sort=&amp;fq=&amp;total=19&amp;recno=3&amp;subType=event">				Quellen und Methoden der Geschichtswissenschaft im digitalen Zeitalter: Neue Zugänge für eine etablierte Diszipli

In [7]:
first_hit = results[0].find('a')['href']
first_hit

'/searching/id/rezbuecher-28479?title=t-kahlert-unternehmungen-grossen-stils&q=rezension grundwissenschaft&sort=&fq=&total=19&recno=1&subType=reb'

In [10]:
link_url = base_url + first_hit
link_url = link_url.replace(" ", "%20")
link_url

'http://www.hsozkult.de/searching/id/rezbuecher-28479?title=t-kahlert-unternehmungen-grossen-stils&q=rezension%20grundwissenschaft&sort=&fq=&total=19&recno=1&subType=reb'

In [12]:
review_html = urlopen(link_url)
bs_review = BeautifulSoup(review_html, 'html.parser')

In [13]:
meta_data = {}

for key, val in bs_review.find_all('div', {'class': 'hfn-item-metarow'}):
    k = key.get_text()
    k = k.strip()
    v = val.get_text()
    v = v.strip()
    v = re.sub(r"[\n\t]", " ", v)
    if k != "":
        meta_data[k] = v
        
meta_data

{'Autor(en)': 'Kahlert, Torsten',
 'Erschienen': 'Berlin  2017:              be.bra Verlag',
 'ISBN': '978-3-95410-089-7',
 'Preis': '€ 40,00',
 'Titel': '„Unternehmungen großen Stils“.  Wissenschaftsorganisation, Objektivität und Historismus im 19. Jahrhundert',
 'Umfang': '384 S.'}

In [15]:
review = bs_review.find('div', {'class': 'hfn-item-fulltext'}).find_all('p')
review_content = [paragraph.get_text() for paragraph in review]
review_content

['Digital history wird diskutiert[1], ausprobiert[2] und – vereinzelt – institutionalisiert.[3] Inwiefern die Auswertung massenhafter Datensätze durch digitale Techniken zur Erweiterung historischen Wissens beiträgt, ist dabei noch offen. Torsten Kahlerts Dissertation, die an der Humboldt-Universität Berlin entstanden und 2017 publiziert worden ist, liest sich vor diesem Hintergrund mehr als interessant. Ausgehend von den Fragen nach ihrer Legitimation, institutionellen Verankerung, Konstruktion, Konsequenz für den Wissenschaftsbetrieb und den Praktiken ihrer Umsetzung untersucht Kahlert drei Massendatenprojekte des 19. Jahrhunderts: Theodor Mommsens Corpus Inscriptionum Latinarum (CIL) (S. 53–184), Rudolf Virchows „Schulkinderuntersuchung“ (S. 185–273) und Gustav Schmollers Acta Borussica (S. 275–319). Alle Fallbeispiele sind begründet ausgewählt, denn ihre Fachrichtungen gehörten zeitgenössisch zu den zentralen Disziplinen, ihre Initiatoren besaßen eine ähnliche Reputation und sie wa

In [16]:
with open('rezension.txt', 'w', encoding='utf-8') as f:
    f.write("Rezension von: {}: {}, {}".format(
        meta_data['Autor(en)'],
        meta_data['Titel'], 
        meta_data['Erschienen']))
    f.write("\n\n")
    f.write("\n".join(review_content))